In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [2]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
# tokenization
def tokenization(text):
  text = text.lower()
  text = text.replace('?',"")
  text = text.replace("'","")
  return text.split()

In [4]:
tokenization(df.iloc[0,0])

['what', 'is', 'the', 'capital', 'of', 'france']

In [5]:
#vocab
vocab = {'<UNK>':0}
def build_vocab(text):
  tokenized_question = tokenization(text['question'])
  tokenized_answer = tokenization(text['answer'])
  merged_text = tokenized_question + tokenized_answer
  for token in merged_text:
    if token in vocab:
      pass
    else:
      vocab[token] = len(vocab)


In [6]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
# text to numerical indices
def text_to_numbers(text, vocab):
  indices_list = []
  for token in tokenization(text):
    if token in vocab:
      indices_list.append(vocab[token])
    else:
      indices_list.append(vocab['<UNK>'])
  return indices_list

In [8]:
text_to_numbers(df.iloc[3,0],vocab)

[1, 2, 3, 17, 18, 19, 20, 21, 22]

In [9]:
class dataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_numbers(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_numbers(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)


In [10]:
newdf = dataset(df,vocab)

In [11]:
newdf[3][0]

tensor([ 1,  2,  3, 17, 18, 19, 20, 21, 22])

In [12]:
dataloader = DataLoader(newdf, batch_size = 1, shuffle = True)

In [13]:
import torch.nn as nn

In [14]:
from typing_extensions import final
class Model(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50,64, batch_first =True)
    self.fc = nn.Linear(64,vocab_size)

  def forward(self, question):
    out = self.embedding(question)
    hidden, final = self.rnn(out)
    out = self.fc(final.squeeze(0))

    return out



In [15]:
lr = 0.001
epochs = 30

In [16]:
model = Model(len(vocab))

In [17]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = lr)

In [18]:
#training
for epoch in range(epochs):
  total_loss = 0
  for question,answer in dataloader:
    optimizer.zero_grad()
    output = model(question)
    loss = loss_function(output,answer[0])
    loss.backward()
    optimizer.step()

    total_loss = total_loss+loss
  print(f'epoch: {epoch+1} Loss:  {total_loss}')

epoch: 1 Loss:  527.0223388671875
epoch: 2 Loss:  455.2736511230469
epoch: 3 Loss:  375.8896789550781
epoch: 4 Loss:  318.7472229003906
epoch: 5 Loss:  270.05181884765625
epoch: 6 Loss:  222.83721923828125
epoch: 7 Loss:  179.66587829589844
epoch: 8 Loss:  141.75103759765625
epoch: 9 Loss:  109.64192199707031
epoch: 10 Loss:  84.43643951416016
epoch: 11 Loss:  65.33582305908203
epoch: 12 Loss:  51.35143280029297
epoch: 13 Loss:  40.81144714355469
epoch: 14 Loss:  32.98897933959961
epoch: 15 Loss:  27.24814224243164
epoch: 16 Loss:  22.517242431640625
epoch: 17 Loss:  19.074947357177734
epoch: 18 Loss:  16.220975875854492
epoch: 19 Loss:  13.91517448425293
epoch: 20 Loss:  11.940413475036621
epoch: 21 Loss:  10.457293510437012
epoch: 22 Loss:  9.150481224060059
epoch: 23 Loss:  8.044564247131348
epoch: 24 Loss:  7.134542465209961
epoch: 25 Loss:  6.370168685913086
epoch: 26 Loss:  5.742514133453369
epoch: 27 Loss:  5.179965972900391
epoch: 28 Loss:  4.677100658416748
epoch: 29 Loss:  4.

In [19]:
#pipeline
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_numbers(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [20]:
predict(model, "How many legs does a spider have?")

8
